In [2]:
''' READ THE OUTPUTS OF THE Centroid, DBSCAN, and home_addresses file and compare, evaluate ! '''

' READ THE OUTPUTS OF THE Centroid, DBSCAN, and home_addresses file and compare, evaluate ! '

In [52]:
city = 'budapest'

import os
import mpu
import pandas as pd

In [53]:
''' USERS HOME LOCATIONS'''


# parse city data
city = 'bristol'
outfolder =  '../ProcessedData/' + city + '/'


def get_users_homes(fn, city, outfolder, num = 3):
    
    users_coordinates = {}

    for line in open(outfolder + fn):
        if num == 4:
            user, lng, lat, venue = line.strip().split('\t')
        else:
            user, lng, lat = line.strip().split('\t')

        users_coordinates[user] = (float(lng), float(lat))
        
    return users_coordinates


def get_groundtruth_homes(city, outfolder):

    return get_users_homes( 'user_info/' + city + '_groundtruth_home_locations_unique.dat', city, outfolder, 4)


In [57]:
def get_homes_from_methods(city, outfolder):
    
    files = ['/user_homes/' + fff for fff in os.listdir(outfolder + '/user_homes/' ) if '_user_homes_' in fff]

    methods_homes = {}
    for fn in files:

        method      = fn.split('homes_')[1].replace('.dat','')
        users_homes = get_users_homes(fn, city, outfolder)

        for user, home in users_homes.items():

            if user not in methods_homes:
                methods_homes[user] = {}

            methods_homes[user][method] = home
    
    return methods_homes
    
    
get_homes_from_methods(city, outfolder)
a = 1

In [58]:
def get_distance_from_groundtruth(city, outfolder):

    homedistances_users_methods = {}

    for user, home in groundtruth_homes.items():

        for method, home_ in methods_homes[user].items():

            if user not in homedistances_users_methods:
                homedistances_users_methods[user] = {}

            homedistances_users_methods[user][method] =  mpu.haversine_distance((home[1], home[0]), (home_[1], home_[0])) 


    return pd.DataFrame.from_dict(homedistances_users_methods, orient = 'index')

    


In [80]:
import matplotlib.pyplot as plt
import numpy as np 


def dummy_viz_goodness_of_dist(distance_from_groundtruth):

    def viz_ax(ax, method, distance_from_groundtruth):

        ax.set_title(method)
        ax.hist(distance_from_groundtruth[method].dropna(), bins = 100, alpha = 0.7)
        ax.set_yscale('log')
        ax.set_xlim([0,5])
        
    f, ax   = plt.subplots(2, 2, figsize=(15, 10))
    methods = distance_from_groundtruth.keys()

    print methods[1]
    viz_ax(ax[0,0], methods[5], distance_from_groundtruth)    
    viz_ax(ax[0,1], methods[1], distance_from_groundtruth)    
    viz_ax(ax[1,0], methods[2], distance_from_groundtruth)    
    viz_ax(ax[1,1], methods[3], distance_from_groundtruth)    

    plt.savefig(outfolder   + '/user_homes/' + city + '_home_distance_from_groundtruth_' + str(LIMIT) + '.png')
    plt.close()
    
    


centroids_cutoff=2.0


In [84]:
groundtruth_homes         = get_groundtruth_homes(city, outfolder)
methods_homes             = get_homes_from_methods(city, outfolder)
distance_from_groundtruth = get_distance_from_groundtruth(city, outfolder)

#print distance_from_groundtruth
df_res = pd.DataFrame()
df_res['Averages'] = distance_from_groundtruth.mean(axis=0)
df_res['Stdevs']   = distance_from_groundtruth.std(axis=0)

df_res.to_csv(outfolder   + '/user_homes/' + city + '_CENTROID_COMPARISON_RES.csv', sep = '\t', float_format='%.3f')

dummy_viz_goodness_of_dist(distance_from_groundtruth)    
  
    


centroids_cutoff=2.0
